In [1]:
import csv, sys, collections
from datetime import datetime
from utils.common import *
from tqdm import tqdm

# Configuration

In [2]:
RAW_TRAIN_PATH = "../tr.r0.csv"
RAW_VAL_PATH = "../va.r0.csv"

# file name after pre-processing step (processed csv file)
PROCESSED_TRAIN_PATH = "processed_train.csv"
PROCESSED_VAL_PATH = "processed_val.csv"

# hashed csv file's name
HASHED_TRAIN_PATH = "hashed_train.csv"
HASHED_VAL_PATH = "hashed_val.csv"

ROWS_FOR_TRAINING = 4 * 1e6
LEARNING_RATE = 0.03
EPOCHS = 13

# removed 'pub_id', 'pub_domain', 'pub_category', added 'app/site_id', 'app/site_domain', 'app/site_category'
FIELDS = ['id','click','hour','banner_pos','device_id','device_ip','device_model','device_conn_type','C14','C17','C20','C21',
         'app_id', 'app_domain', 'app_category', 'site_id', 'site_domain', 'site_category']
NEW_FIELDS = FIELDS+['device_id_count','device_ip_count','user_count','smooth_user_hour_count','user_click_histroy']

# Calculate counting features

In [3]:
def scan(path, is_train):
    '''
    copy from base/util/gen_data.py
    '''
    for i, row in tqdm(enumerate(csv.DictReader(open(path)), start=1)):
        if i >= ROWS_FOR_TRAINING and is_train:
            break
        user = def_user(row)
        id_cnt[row['device_id']] += 1
        ip_cnt[row['device_ip']] += 1
        user_cnt[user] += 1
        user_hour_cnt[user+'-'+row['hour']] += 1

In [4]:
start = datetime.now()
print('Start: {0}'.format(str(start)))

id_cnt = collections.defaultdict(int)
ip_cnt = collections.defaultdict(int)
user_cnt = collections.defaultdict(int)
user_hour_cnt = collections.defaultdict(int)

scan(RAW_TRAIN_PATH, True)
scan(RAW_VAL_PATH, False)
print('End: {0}, Elapsed time: {1}'.format(
        str(datetime.now()),
        str(datetime.now() - start))
     )

2994it [00:00, 28625.04it/s]

Start: 2020-02-09 16:00:11.189515


3996164it [00:56, 48860.26it/s]
0it [00:00, ?it/s]
4860it [00:00, 48599.70it/s]
8939it [00:00, 45958.86it/s]
16428it [00:00, 51982.38it/s]
24601it [00:00, 58352.82it/s]
31143it [00:00, 60305.27it/s]
36693it [00:00, 57581.74it/s]
44129it [00:00, 61761.40it/s]
50851it [00:00, 63302.86it/s]
57099it [00:00, 57961.75it/s]
62925it [00:01, 52588.27it/s]
68301it [00:01, 48985.63it/s]
75535it [00:01, 54235.40it/s]
82036it [00:01, 57071.20it/s]
87970it [00:01, 51930.17it/s]
93411it [00:01, 45686.99it/s]
98290it [00:01, 42228.61it/s]
103093it [00:01, 43815.58it/s]
111000it [00:02, 50579.35it/s]
116582it [00:02, 49837.51it/s]
122321it [00:02, 51884.86it/s]
127790it [00:02, 50026.52it/s]
135819it [00:02, 56403.96it/s]
144430it [00:02, 62914.33it/s]
152117it [00:02, 66537.06it/s]
160400it [00:02, 70709.51it/s]
167843it [00:02, 66085.92it/s]
174773it [00:03, 49292.05it/s]
180553it [00:03, 50862.85it/s]
188660it [00:03, 57263.43it/s]
196619it [00:03, 62523.98it/s]
203499it [00:03, 55960.63it/s]
210400

1707353it [00:26, 67548.37it/s]
1714147it [00:26, 66141.73it/s]
1720877it [00:26, 66482.63it/s]
1727550it [00:26, 65497.81it/s]
1734620it [00:27, 66975.47it/s]
1741340it [00:27, 67039.48it/s]
1748264it [00:27, 67684.40it/s]
1755045it [00:27, 67214.12it/s]
1761776it [00:27, 62807.19it/s]
1768122it [00:27, 59918.76it/s]
1774188it [00:27, 57857.76it/s]
1780041it [00:27, 57274.65it/s]
1786020it [00:27, 58005.24it/s]
1793970it [00:28, 63124.61it/s]
1801980it [00:28, 67409.73it/s]
1809825it [00:28, 70380.41it/s]
1817586it [00:28, 72403.18it/s]
1824959it [00:28, 72166.70it/s]
1832269it [00:28, 70212.06it/s]
1839367it [00:28, 69006.89it/s]
1846386it [00:28, 69356.75it/s]
1853625it [00:28, 70237.43it/s]
1860791it [00:28, 70656.61it/s]
1868638it [00:29, 72831.03it/s]
1876201it [00:29, 73648.28it/s]
1884101it [00:29, 75174.41it/s]
1892115it [00:29, 76596.86it/s]
1899799it [00:29, 72497.85it/s]
1907112it [00:29, 72338.21it/s]
1914390it [00:29, 72062.90it/s]
1922154it [00:29, 73649.96it/s]
1929550i

3188994it [00:52, 65203.77it/s]
3195518it [00:52, 61989.20it/s]
3201753it [00:52, 59923.78it/s]
3207786it [00:53, 56738.03it/s]
3213523it [00:53, 52842.04it/s]
3218906it [00:53, 51849.61it/s]
3224164it [00:53, 51837.94it/s]
3229399it [00:53, 50561.66it/s]
3235157it [00:53, 52479.41it/s]
3242214it [00:53, 56850.48it/s]
3248039it [00:53, 55873.27it/s]
3253796it [00:53, 56369.70it/s]
3259505it [00:54, 54876.30it/s]
3265052it [00:54, 53812.49it/s]
3271271it [00:54, 56076.94it/s]
3276936it [00:54, 44587.81it/s]
3282562it [00:54, 47545.35it/s]
3289055it [00:54, 51696.79it/s]
3296056it [00:54, 56098.28it/s]
3302335it [00:54, 57947.28it/s]
3309062it [00:54, 60458.38it/s]
3315319it [00:55, 55595.83it/s]
3321104it [00:55, 52197.12it/s]
3327762it [00:55, 55812.73it/s]
3334946it [00:55, 59814.55it/s]
3342199it [00:55, 63133.86it/s]
3349438it [00:55, 65650.48it/s]
3356655it [00:55, 67477.83it/s]
3363807it [00:55, 68639.53it/s]
3371039it [00:55, 69702.11it/s]
3378082it [00:56, 69312.57it/s]
3385065i

End: 2020-02-09 16:02:23.858922, Elapsed time: 0:02:12.669485


# Add counting features & history features to new csv file 

In [5]:
def gen_data(src_path, dst_path, is_train):
    '''
    copy from base/util/gen_data.py
    '''
    reader = csv.DictReader(open(src_path))
    writer = csv.DictWriter(open(dst_path, 'w'), NEW_FIELDS)
    writer.writeheader()

    for i, row in tqdm(enumerate(reader, start=1)):
        if i >= ROWS_FOR_TRAINING and is_train:
            break
        new_row = {}
        for field in FIELDS:
            new_row[field] = row[field]

        new_row['device_id_count'] = id_cnt[row['device_id']]
        new_row['device_ip_count'] = ip_cnt[row['device_ip']]

        user, hour = def_user(row), row['hour']
        new_row['user_count'] = user_cnt[user]
        new_row['smooth_user_hour_count'] = str(user_hour_cnt[user+'-'+hour])

        if has_id_info(row):

            if history[user]['prev_hour'] != row['hour']:
                history[user]['history'] = (history[user]['history'] + history[user]['buffer'])[-4:]
                history[user]['buffer'] = ''
                history[user]['prev_hour'] = row['hour']

            new_row['user_click_histroy'] = history[user]['history']

            if is_train:
                history[user]['buffer'] += row['click']
        else:
            new_row['user_click_histroy'] = ''
            
        writer.writerow(new_row)

In [6]:
start = datetime.now()
print('Start: {0}'.format(str(start)))

history = collections.defaultdict(lambda: {'history': '', 'buffer': '', 'prev_hour': ''})

gen_data(src_path=RAW_TRAIN_PATH, dst_path=PROCESSED_TRAIN_PATH,is_train=ROWS_FOR_TRAINING != -1)
gen_data(src_path=RAW_VAL_PATH, dst_path=PROCESSED_VAL_PATH, is_train=False)

print('End: {0}, Elapsed time: {1}'.format(
        str(datetime.now()),
        str(datetime.now() - start))
     )


0it [00:00, ?it/s]
2564it [00:00, 25636.11it/s]

Start: 2020-02-09 16:02:23.918442



4999it [00:00, 25234.75it/s]
7370it [00:00, 24755.95it/s]
10349it [00:00, 26077.37it/s]
13291it [00:00, 26996.78it/s]
16216it [00:00, 27635.12it/s]
19299it [00:00, 28521.10it/s]
22516it [00:00, 29524.58it/s]
25611it [00:00, 29937.88it/s]
28573it [00:01, 29840.01it/s]
31498it [00:01, 28100.34it/s]
34452it [00:01, 28514.56it/s]
37291it [00:01, 28363.07it/s]
40272it [00:01, 28780.00it/s]
43291it [00:01, 29186.95it/s]
46345it [00:01, 29579.76it/s]
49445it [00:01, 29990.66it/s]
52517it [00:01, 30205.70it/s]
55539it [00:01, 30186.30it/s]
58559it [00:02, 29777.38it/s]
61539it [00:02, 29657.43it/s]
64534it [00:02, 29743.53it/s]
67510it [00:02, 25292.54it/s]
70512it [00:02, 26546.28it/s]
73265it [00:02, 26713.78it/s]
76130it [00:02, 27266.36it/s]
79039it [00:02, 27787.85it/s]
82057it [00:02, 28463.59it/s]
84935it [00:03, 27223.52it/s]
87691it [00:03, 27046.73it/s]
90512it [00:03, 27381.53it/s]
93634it [00:03, 28428.69it/s]
96538it [00:03, 28607.42it/s]
99670it [00:03, 29368.38it/s]
102764it [0

748869it [00:25, 29168.33it/s]
751976it [00:25, 29710.18it/s]
755158it [00:25, 30311.94it/s]
758346it [00:25, 30764.04it/s]
761573it [00:25, 31198.62it/s]
764742it [00:25, 31344.15it/s]
767892it [00:25, 31057.96it/s]
771010it [00:26, 30916.78it/s]
774160it [00:26, 31087.28it/s]
777288it [00:26, 31143.26it/s]
780407it [00:26, 30962.33it/s]
783653it [00:26, 31395.56it/s]
786876it [00:26, 31638.88it/s]
790043it [00:26, 31179.00it/s]
793322it [00:26, 31644.61it/s]
796491it [00:26, 30979.42it/s]
799672it [00:26, 31223.62it/s]
802835it [00:27, 31342.49it/s]
806026it [00:27, 31509.42it/s]
809226it [00:27, 31653.29it/s]
812394it [00:27, 31473.98it/s]
815553it [00:27, 31507.38it/s]
818705it [00:27, 28197.57it/s]
821591it [00:27, 21622.07it/s]
824032it [00:27, 20284.92it/s]
826272it [00:28, 17728.61it/s]
828251it [00:28, 16376.62it/s]
830054it [00:28, 14824.92it/s]
831680it [00:28, 13476.51it/s]
834399it [00:28, 15878.69it/s]
837456it [00:28, 18552.73it/s]
840518it [00:28, 21040.14it/s]
843618it

1476158it [00:50, 30439.61it/s]
1479203it [00:50, 30415.98it/s]
1482245it [00:50, 29598.83it/s]
1485337it [00:50, 29980.81it/s]
1488473it [00:50, 30381.65it/s]
1491619it [00:50, 30696.42it/s]
1494777it [00:50, 30951.99it/s]
1497889it [00:50, 31000.71it/s]
1501008it [00:50, 31056.80it/s]
1504152it [00:51, 31170.39it/s]
1507305it [00:51, 31275.37it/s]
1510434it [00:51, 31085.51it/s]
1513544it [00:51, 30640.69it/s]
1516660it [00:51, 30793.64it/s]
1519879it [00:51, 31199.22it/s]
1523052it [00:51, 31355.92it/s]
1526281it [00:51, 31628.10it/s]
1529457it [00:51, 31665.58it/s]
1532625it [00:51, 31205.14it/s]
1535748it [00:52, 30234.39it/s]
1538780it [00:52, 30024.14it/s]
1541869it [00:52, 30276.60it/s]
1544979it [00:52, 30517.58it/s]
1548067it [00:52, 30625.14it/s]
1551133it [00:52, 30188.36it/s]
1554267it [00:52, 30524.54it/s]
1557531it [00:52, 31129.63it/s]
1560787it [00:52, 31544.35it/s]
1563947it [00:52, 31299.74it/s]
1567086it [00:53, 31326.75it/s]
1570228it [00:53, 31354.03it/s]
1573366i

...

2353237it [01:22, 31397.33it/s]

2356388it [01:22, 31430.38it/s]

2359651it [01:22, 31778.59it/s]

2362871it [01:22, 31901.28it/s]

2366164it [01:22, 32202.80it/s]

2369393it [01:22, 32226.71it/s]

2372619it [01:23, 31908.68it/s]

2375813it [01:23, 31827.22it/s]

2378998it [01:23, 31579.13it/s]

2382352it [01:23, 32141.66it/s]

2385570it [01:23, 31816.15it/s]

2388904it [01:23, 32257.61it/s]

2392134it [01:23, 31957.96it/s]

2395365it [01:23, 32061.91it/s]

2398606it [01:23, 32164.28it/s]

2401825it [01:23, 32069.30it/s]

2405149it [01:24, 32410.43it/s]

2408392it [01:24, 32292.90it/s]

2411623it [01:24, 31976.02it/s]

2414823it [01:24, 30506.69it/s]

2417890it [01:24, 30555.31it/s]

2420957it [01:24, 30337.35it/s]

2423999it [01:24, 30252.48it/s]

2427030it [01:24, 29712.56it/s]

2430121it [01:24, 30061.40it/s]

2433133it [01:25, 28199.88it/s]

2435981it [01:25, 26509.91it/s]

2439141it [01:25, 27854.76it/s]

2441973it [01:25, 25773.78it/s]

2444705it [01:25, 26218.42it/s]

2447390it 

2966442it [01:44, 32540.14it/s]

2969703it [01:45, 32559.71it/s]

2972962it [01:45, 31499.29it/s]

2976299it [01:45, 32037.74it/s]

2979569it [01:45, 32233.21it/s]

2982934it [01:45, 32644.12it/s]

2986205it [01:45, 31925.69it/s]

2989510it [01:45, 32253.35it/s]

2992985it [01:45, 32963.74it/s]

2996415it [01:45, 33351.92it/s]

2999757it [01:45, 32434.30it/s]

3003011it [01:46, 32443.21it/s]

3006263it [01:46, 32175.78it/s]

3009694it [01:46, 32786.23it/s]

3012980it [01:46, 32512.47it/s]

3016237it [01:46, 32512.95it/s]

3019492it [01:46, 32153.93it/s]

3022711it [01:46, 32027.34it/s]

3026110it [01:46, 32591.73it/s]

3029533it [01:46, 33066.31it/s]

3032890it [01:46, 33212.96it/s]

3036215it [01:47, 32483.62it/s]

3039470it [01:47, 31762.75it/s]

3042805it [01:47, 32222.15it/s]

3046035it [01:47, 32153.61it/s]

3049318it [01:47, 32353.18it/s]

3052558it [01:47, 31696.68it/s]

3055734it [01:47, 27507.24it/s]

3059054it [01:47, 28997.37it/s]

3062371it [01:47, 30134.03it/s]

3065507it 

3647415it [02:06, 32173.62it/s]

3650678it [02:06, 32307.07it/s]

3653914it [02:07, 31887.94it/s]

3657281it [02:07, 32396.84it/s]

3660563it [02:07, 32521.38it/s]

3663856it [02:07, 32642.82it/s]

3667139it [02:07, 31457.00it/s]

3670296it [02:07, 30303.58it/s]

3673394it [02:07, 30499.42it/s]

3676478it [02:07, 30596.88it/s]

3679783it [02:07, 31291.66it/s]

3682923it [02:07, 31283.18it/s]

3686059it [02:08, 31217.92it/s]

3689394it [02:08, 31825.74it/s]

3692606it [02:08, 31911.65it/s]

3695802it [02:08, 31586.29it/s]

3699004it [02:08, 31714.39it/s]

3702179it [02:08, 31461.36it/s]

3705588it [02:08, 32205.56it/s]

3708815it [02:08, 31962.23it/s]

3712197it [02:08, 32495.97it/s]

3715453it [02:08, 31986.66it/s]

3718658it [02:09, 31602.43it/s]

3721959it [02:09, 32006.44it/s]

3725165it [02:09, 31849.66it/s]

3728527it [02:09, 32360.61it/s]

3731768it [02:09, 32115.26it/s]

3734984it [02:09, 32036.22it/s]

3738191it [02:09, 31917.26it/s]

3741474it [02:09, 32184.86it/s]

3744743it 

4333393it [02:29, 32522.06it/s]

4336652it [02:29, 32130.07it/s]

4339871it [02:29, 32105.11it/s]

4343086it [02:29, 31156.21it/s]

4346286it [02:29, 31404.45it/s]

4349562it [02:29, 31798.18it/s]

4353041it [02:29, 32639.35it/s]

4356349it [02:29, 32768.56it/s]

4359633it [02:30, 32669.18it/s]

4363003it [02:30, 32970.46it/s]

4366369it [02:30, 33172.97it/s]

4369690it [02:30, 33003.42it/s]

4372998it [02:30, 33025.97it/s]

4376351it [02:30, 33174.88it/s]

4379692it [02:30, 33244.87it/s]

4383018it [02:30, 32806.16it/s]

4386386it [02:30, 33062.55it/s]

4389695it [02:30, 33067.60it/s]

4393005it [02:31, 33075.24it/s]

4396487it [02:31, 33579.62it/s]

4399848it [02:31, 33504.56it/s]

4403213it [02:31, 33547.25it/s]

4406569it [02:31, 33255.15it/s]

4409896it [02:31, 32733.50it/s]

4413195it [02:31, 32809.67it/s]

4416479it [02:31, 32326.80it/s]

4419937it [02:31, 32968.49it/s]

4423240it [02:31, 32684.65it/s]

4426710it [02:32, 33262.45it/s]

4430042it [02:32, 32798.92it/s]

4433444it 

End: 2020-02-09 16:07:12.275059, Elapsed time: 0:04:48.356702


# Paralized hashing PROCESSED_TRAIN/VAL_PATH and save to HASHED_TRAIN/VAL_PATH

In [7]:
nr_thread = 12

In [8]:
# split processed file into nr_thread csv_files
split(path=PROCESSED_TRAIN_PATH, nr_thread=nr_thread)
split(path=PROCESSED_VAL_PATH, nr_thread=nr_thread)

In [9]:
# parallelly hashing splited csv_files and save to nr_thread hashed csv_files
parallel_convert(
    "utils/2.py", 
    [PROCESSED_TRAIN_PATH, PROCESSED_VAL_PATH, HASHED_TRAIN_PATH, HASHED_VAL_PATH], 
    nr_thread)

In [10]:
# delete old splited processed files
delete(PROCESSED_TRAIN_PATH, nr_thread)
delete(PROCESSED_VAL_PATH, nr_thread)

# merge nr_thread hashed csv_files into 1 file
cat(HASHED_TRAIN_PATH, nr_thread)
cat(HASHED_VAL_PATH, nr_thread)

# delete old splited hashed csv_files
delete(HASHED_TRAIN_PATH, nr_thread)
delete(HASHED_VAL_PATH, nr_thread)

In [11]:
no_thread = 1 # number of thread for training
cmd = '../base/mark1 -r {0} -s {1} -t {2} {3} {4}'.format(LEARNING_RATE, no_thread, EPOCHS, HASHED_VAL_PATH, HASHED_TRAIN_PATH)
subprocess.call(cmd.split())

0